In [1]:
%load_ext autoreload
%autoreload 2
import ase
from ase.visualize import view
from ase.build import molecule
from ase.io import Trajectory, write, read
from ase.units import mol # Avogadro Constant
# calculator
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.units import kB, fs, Pascal

import numpy as np

def vis(atoms):
    v = view(atoms, viewer='ngl')
    v.view.add_representation("ball+stick")
    display(v)

#estimator = Estimator(calc_mode='CRYSTAL_PLUS_D3')
estimator = Estimator(model_version="v2.0.0", max_retries=30,  calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3)
calculator = ASECalculator(estimator)

from ase.constraints import FixAtoms

print(f"pfp_api_client: {pfp_api_client.__version__}")
print(f"Estimator model version: {estimator.model_version}")
print("available versions: ", estimator.available_models)

pfp_api_client: 1.15.0
Estimator model version: v2.0.0
available versions:  ['latest', 'v0.0.0', 'v1.0.0', 'v1.1.0', 'v2.0.0', 'v3.0.0', 'v4.0.0', 'v5.0.0', 'v6.0.0']


In [2]:
import os
import sys
sys.path.append(os.path.join('..'))

from lammpslib3 import LAMMPSlib
from mixing_mod import SumCalculator

# Example of hybrid/overlay

cmds = ["pair_style hybrid/overlay lj/cut 12.0",
        "pair_coeff * * lj/cut 0 0.1",]

# Long range for water
cmdLR = ["pair_style hybrid/overlay lj/cut 12.0 lj/cut/coul/long 20.0 20.0",
         "pair_coeff * * lj/cut 0 0",
         "pair_coeff 4 4 lj/cut/coul/long 0 0",
         "kspace_style ewald 1.0e-4",]

lammpsLR = LAMMPSlib(lmpcmds=cmdLR,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0',],
                   atom_types={'C':1, 'H':2, 'O':3, 'F':4}, keep_alive=True)

lammps1 = LAMMPSlib(lmpcmds=cmds,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0'],
                   atom_types={'C':1, 'H':2, 'O':3, 'F':4}, keep_alive=True)
lammps2 = LAMMPSlib(lmpcmds=cmds,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0'],
                   atom_types={'C':1, 'H':2, 'O':3, 'F':4}, keep_alive=True)

pfp1 = ASECalculator(Estimator(model_version="v2.0.0", max_retries=30,  calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3))
pfp2 = ASECalculator(Estimator(model_version="v2.0.0", max_retries=30,  calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3))

In [3]:
from simpleQMMM_lammps_charge import SimpleQMMM_charge

atoms = read("opt_graphene_oxide2_h2o_5nm_final.xyz")

CHARGE_VALUE=0.0
#CHARGE_VALUE=0.00

atnum = atoms.get_atomic_numbers()
#print(atnum)
qm_idx = np.arange(len(atoms))
qm_idx = qm_idx[atnum != 6]
charge_idx=range(288)
charge_values = np.zeros_like(charge_idx, dtype="float")

# 72 C atoms for GO layer -> charged +q (:72) (1)
# 72 C layer (total 72+72=144 atoms) (2)
# 72 C atoms for GO layer -> charged -q (144:216) (3)
# 72 C layer (total 216+72=288 atoms) (4)
# ==== mm/qm layer
# 10 O + 7 OH = 24 atoms (total 288+24=312) on (1)
# 10 O + 7 OH = 24 atoms (total 312+24=336) on (3)

# CAUTION the order of cathode / anode is different from the graphene+h2o calculation
charge_values[:72] = -CHARGE_VALUE
charge_values[144:216] = CHARGE_VALUE

pfp_plus_LR = SumCalculator([calculator, lammpsLR], ignore_not_implemented=True)

atoms.set_calculator( SimpleQMMM_charge(qm_idx,
                    pfp_plus_LR,
                    lammps1,
                    lammps2,
                    charge_idx,
                    charge_values,
                    vacuum=None,  # if None, QM cell = MM cell
                    mmcalc1_2 = pfp1,
                    mmcalc2_2 = pfp2
                    ) )

print(atoms.get_potential_energy())
np.savetxt("force_ewald_NPT.txt",atoms.get_forces())

-6171.812333972595


In [4]:
from ase.md.npt import NPT
#from ase.md import Langevin
import time
time_sta = time.perf_counter()
time_pre = time_sta

T = 350.0
P = 100000*Pascal
dt = 0.25*fs

tdamp = 100*dt
pdamp = pdamp = 2*0.6*(75*fs)**2.0 

#lmp = lammps()

OUT_traj = "NPT_ewald.traj"
OUT_log = "NPT_ewald.log"


MaxwellBoltzmannDistribution(atoms, temperature_K = T)
Stationary(atoms)
atoms.get_temperature()

dyn = NPT(atoms, dt, temperature_K = T, externalstress = P, ttime = tdamp, pfactor = pdamp,  trajectory = OUT_traj, mask=[[1,0,0],[0,1,0],[0,0,1]], loginterval=100)

#dyn = Langevin(atoms=atoms, timestep=dt, 
#               temperature_K = T, friction=0.01, trajectory = OUT_traj, loginterval=100)


Header= "step etotal T e_pot stress Volume density"
print("NPT Simulation")
print(Header)

weight = np.sum(atoms.get_masses())*1.673e-27

file = open(OUT_log, "w")
file.write(Header + "\n")
def print_dyn():
    global time_pre
    V = atoms.get_volume()*1E-30 # in m^3
    density = weight/V
    time_now = time.perf_counter()
    #line = f"{dyn.get_number_of_steps(): >3} {atoms.get_total_energy():.6f} {atoms.get_temperature():.6f} {atoms.get_potential_energy():.6f} {-atoms.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f} {time_now-time_sta:.3f} {time_now-time_pre:.3f}"
    line = f"{dyn.get_number_of_steps(): >3} {atoms.get_total_energy():.6f} {atoms.get_temperature():.6f} {atoms.get_potential_energy():.6f} {-atoms.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f} {V:8.6e} {density:8.6e}"
    time_pre = time_now
    file.write(line+"\n")
    print(line, end = "\r")
    
    
dyn.attach(print_dyn, interval=50)

#steps = 600000
steps = 200
dyn.run(steps)


file.close()

NPT Simulation
step etotal T e_pot stress Volume density


In [5]:
# test visualization

from ase.io import read,write

OUT_traj = "NPT_ewald.traj"
test_traj = read(OUT_traj, index=":")
out_xyz = "NPT_ewald.xyz"

for iframe, atoms in enumerate(test_traj):
    
    if iframe ==0:
        atoms.wrap()
        write(out_xyz, atoms)
    else:
        write(out_xyz, atoms, append=True)        
    
    #print(atoms.get_volume())
#vis(test_traj)

